# Logística de envíos: ¿Cuándo llega?

## Mentoría DiploDatos 2019 

### Integrantes:

- Alini, Walter
- Frau, Johanna
- Salina, Noelia

### Mentora:

- Dal Lago, Virginia

### Práctico: Análisis exploratorio y curación

## Motivación

En la actualidad, cada vez más productos se comercializan a través de una plataforma online. Una de las principales ventajas de este sistema es que el usuario puede recibir el producto en su domicilio en una fecha determinada. Pero, ¿cómo sabemos qué día va a llegar? ¿A partir de qué datos podemos predecir la demora del envío? En este práctico se trabajará con datos de envíos de MercadoLibre, el e-commerce más grande de Latinoamérica, analizando y modelando el problema de logística de envíos para poder responder ¿cuándo llega?

## Descripción del dataset

**Datos:**  El conjunto de datos seleccionado para realizar el práctico corresponde a un muestreo aleatorio no uniforme de 500.000 envíos de MercadoLibre. Estos envíos fueron realizados en Brasil en el período comprendido entre Octubre de 2018 y Abril de 2019 (las fechas originales han sido modificadas y adaptadas a un período de tiempo diferente, conservando el día de la semana y considerando los feriados correspondientes). Mientras que las fechas han sido modificadas, los horarios registrados en el dataset son los originales. Los datos comprenden variables tanto categóricas como numéricas. 

El dataset cuenta con las siguientes columnas:

- **Sender_state:** Estado de Brasil de donde sale el envío.
- **Sender_zipcode:** Código postal (de 5 dígitos) de donde sale el envío.
- **Receiver_state:** Estado de Brasil a donde llega el envío.
- **Receiver_zipcode:** Código postal (de 5 dígitos) a donde llega el envío.
- **Shipment_type:** Método de envío (normal, express, super).
- **Quantity:** Cantidad de productos en un envío.
- **Service:** Servicio del correo con el cual se realizó un envío.
- **Status:** Estado del envío (set: listo para ser enviado, sent: enviado, done: entregado, failed: no entregado, cancelled: cancelado).
- **Date_created:** Fecha de creación del envío.
- **Date_sent:** Fecha y hora en que se realizó el envío (salió del correo).
- **Date_visit:** Fecha y hora en que se entregó el envío al destinatario.
- **Shipment_days:** Días hábiles entre que el envío fue enviado (salió del correo) y que fue entregado.

## Objetivos generales


  * Realizar un estudio exploratorio del dataset para extraer información útil sobre el problema a resolver
  * Desarrollar visión crítica en relación a la problemática para llevar a cabo el procedimiento de ciencia de datos
  * Desarrollar habilidades de comunicación de la información obtenida a partir de los datos de manera clara y sencilla.


## Objetivos específicos

ToDo habría que agregar objetivos especificos una vez que tengamos mas avanazado el practico como sugirio la vir en las corecciones

## Metodología

A lo largo de este trabajo trataremos de responder a las siguientes preguntas:

1. ¿Existen datos duplicados? ¿Con qué información deberíamos contar para poder determinar con certeza que dichos datos están duplicados y no corresponden a dos envíos diferentes?
1. En relación a las diferentes columnas de fechas, en el práctico anterior notamos que muchas de ellas no eran consistentes. Identificar y cuantificar los valores atípicos de cada columna. ¿Qué tipo de anomalías hay en el conjunto completo? ¿Qué deberíamos realizar con dichos datos?
1. Una información que, a futuro, podría ser relevante para modelar es el estado de Brasil al cual llega el envío. Sin embargo, esta información está codificada en la columna en formato de texto. ¿De qué maneras podríamos transformar la misma en valores numéricos para utilizar en un modelo? Mostrar al menos dos ejemplos.
1. A la hora de determinar la promesa de entrega de un envío (fecha estimada de llegada), ¿cuáles son los features que consideran pueden tener mayor relevancia? ¿Cuál es el valor a predecir?
1. Suponiendo que queremos emplear un modelo kNN para calcular la promesa de entrega de un envío. ¿Qué transformaciones sugieren realizar sobre los features antes seleccionados? Mostrar al menos un ejemplo.
1. Nos interesa determinar si un envío llegará entre 0-1 días, 2-3 días, 4-5 días, 6-7 días, 8-9 días ó 10 o más días. Construir una nueva columna en el dataset que indique a cuál de estos grupos pertence cada envío, y codificarla numéricamente, de manera tal de poder ser utilizada como valor a predecir. Seleccionar los dos features que se consideren de mayor relevancia para un modelo, y aplicar un procedimiento de clustering (separando en los grupos antes definidos).

Esta comunicación debe estar dirigida para un público técnico pero que desconoce los aspectos propios del problema a resolver (por ejemplo, sus compañeros de clase). Se evaluará, principalmente, la claridad del mensaje presentado, el uso de las herramientas y los conceptos desarrollados en las clases teóricas. 
Además se debe realizar una breve comunicación en pdf (2 páginas máximo) dirigida a un stakeholder del proyecto (por ejemplo, manager), comentando los hallazgos y problemas encontrados, y las posibles acciones a tomar.

## Desarrollo

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy import special
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce # usar "pip install category_encoders" para instalar

BLUE = '#35A7FF'
RED = '#FF5964'
GREEN = '#6BF178'
YELLOW = '#FFE74C'

RELATIVE_PATH = '../diplodatos_2019/mentoria_envios/'
DATA_FILE = 'dataset_sample_corrected.csv'

# Establecemos una semilla por cuestiones de reproducibilidad
np.random.seed(0)

### Lectura y análisis inicial de los datos

In [2]:
ds = pd.read_csv(RELATIVE_PATH + DATA_FILE, 
                       dtype={'sender_zipcode':'int64',
                              'receiver_zipcode':'int64',
                              'quantity':'int64',
                              'service':'int64'},
                       parse_dates=['date_created','date_sent','date_visit'])

In [3]:
ds.columns

Index(['sender_state', 'sender_zipcode', 'receiver_state', 'receiver_zipcode',
       'shipment_type', 'quantity', 'service', 'status', 'date_created',
       'date_sent', 'date_visit', 'shipment_days'],
      dtype='object')

In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 12 columns):
sender_state        500000 non-null object
sender_zipcode      500000 non-null int64
receiver_state      500000 non-null object
receiver_zipcode    500000 non-null int64
shipment_type       500000 non-null object
quantity            500000 non-null int64
service             500000 non-null int64
status              500000 non-null object
date_created        500000 non-null datetime64[ns]
date_sent           499971 non-null datetime64[ns]
date_visit          499398 non-null datetime64[ns]
shipment_days       499398 non-null float64
dtypes: datetime64[ns](3), float64(1), int64(4), object(4)
memory usage: 45.8+ MB


In [5]:
np.random.seed(0) #con propositos de reproducibilidad
ds.sample(10)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
194257,SP,3734,RS,96540,standard,1,4,done,2019-02-17,2019-02-22 03:49:21,2019-03-07 15:09:00,6.0
396319,SP,6422,SP,9580,express,1,5,done,2019-03-04,2019-03-07 20:24:02,2019-03-07 10:02:09,0.0
138738,SP,14940,CE,62650,standard,1,1,done,2019-02-08,2019-02-13 13:51:00,2019-03-11 14:16:00,15.0
66570,SP,1004,MG,32340,express,1,4,done,2019-03-16,2019-03-19 00:52:07,2019-03-25 19:00:00,4.0
149424,SP,13254,BA,44700,express,1,0,done,2019-03-02,2019-03-04 15:41:00,2019-03-12 15:19:00,3.0
128165,SP,9850,SP,12301,express,2,0,done,2019-03-17,2019-03-19 13:39:00,2019-03-20 18:59:00,1.0
264495,SP,3627,BA,44690,standard,1,1,done,2019-03-04,2019-03-06 12:13:00,2019-03-22 14:17:00,11.0
30371,SP,13456,SP,5268,express,2,0,done,2019-03-12,2019-03-13 15:07:00,2019-03-14 16:26:00,1.0
456998,SP,3004,SC,89300,standard,1,3,done,2019-02-15,2019-02-17 04:52:00,2019-03-07 17:56:00,10.0
32001,SP,19013,SP,14815,standard,2,1,done,2019-03-21,2019-03-22 14:32:00,2019-03-26 13:39:00,2.0


###  Shipment_days anómalos

En el práctico anterior habiamos tomado la decisión de no trabajar con valores de la variable **shipment_days** negativos teniendo en cuenta que no los podemos considerar valores reales y que la proporción de estos datos dentro del conjunto total es muy chica comparada con la cantidad total.

Por lo tanto nuestra primer medida será aplicar este filtro.

In [6]:
m_shipment_days_negative = ds.shipment_days < 0
m_shipment_days_positive = ~ m_shipment_days_negative

ds[m_shipment_days_negative].describe()

,sender_zipcode,receiver_zipcode,quantity,service,shipment_days
count,48.00000,48.000000,48.000000,48.000000,48.000000
mean,6933.50000,31424.875000,1.354167,6.770833,-61.083333
std,4507.20175,32577.232988,0.729021,2.398932,21.236093
min,1133.00000,1021.000000,1.000000,2.000000,-77.000000
25%,3111.25000,4705.000000,1.000000,4.000000,-77.000000
50%,6422.00000,18201.000000,1.000000,7.000000,-74.000000
75%,9140.00000,53934.250000,1.000000,9.000000,-37.000000
max,18070.00000,95300.000000,4.000000,9.000000,-5.000000


In [7]:
np.random.seed(0)
ds[m_shipment_days_positive].sample(7)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
111099,SP,14403,SP,13070,express,1,0,done,2019-03-03,2019-03-04 13:35:00,2019-03-08 13:54:00,1.0
184520,SP,14405,BA,40255,standard,1,1,done,2019-02-16,2019-02-20 18:36:00,2019-03-27 11:01:00,22.0
144154,SP,13481,SP,11730,express,1,4,done,2019-03-04,2019-03-07 14:09:56,2019-03-11 17:00:00,2.0
159205,SP,13617,MT,78450,standard,1,1,done,2019-02-26,2019-02-27 09:29:00,2019-03-14 08:54:00,8.0
296683,SP,3679,PI,64200,standard,1,1,done,2019-02-20,2019-02-21 14:26:00,2019-03-01 10:56:00,6.0
497673,SP,6422,AM,69042,express,1,0,done,2019-03-18,2019-03-21 10:40:00,2019-03-27 11:17:00,4.0
387920,SP,7411,PE,55012,standard,2,2,done,2019-02-14,2019-02-18 17:00:00,2019-02-28 08:56:00,8.0


In [8]:
dataset = ds[m_shipment_days_positive]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499952 entries, 0 to 499999
Data columns (total 12 columns):
sender_state        499952 non-null object
sender_zipcode      499952 non-null int64
receiver_state      499952 non-null object
receiver_zipcode    499952 non-null int64
shipment_type       499952 non-null object
quantity            499952 non-null int64
service             499952 non-null int64
status              499952 non-null object
date_created        499952 non-null datetime64[ns]
date_sent           499923 non-null datetime64[ns]
date_visit          499350 non-null datetime64[ns]
shipment_days       499350 non-null float64
dtypes: datetime64[ns](3), float64(1), int64(4), object(4)
memory usage: 49.6+ MB


### Datos duplicados

A partir de la información que tenemos del dataset (ver "Descripción del dataset") no se puede identificar una columna que pueda ser tomada como valor único para una fila del dataset. La única columna que quizá pueda tomar esa función, sería **date_created**. Pero, como vimos en el práctico anterior, esta variable sólo tiene fecha (y no hora) de la creación del evento, con lo cual no tiene sentido tomarla como "id" en nuestro dataset.

Con lo cual, una siguiente hipótesis sería tomar a todos los datos del dataset como el identificador de un envío en particular. Veamos:

In [9]:
ds[ds.duplicated(keep=False)].sort_values('date_created', ascending=True)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
159612,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
172193,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
436591,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
429821,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
133877,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
294562,SP,6422,PR,82600,express,1,0,done,2018-12-28,NaT,NaT,NaN
434070,SP,6422,PR,82600,express,1,0,done,2018-12-28,NaT,NaT,NaN
297183,SP,7143,RS,93042,standard,1,4,done,2019-01-16,2019-02-20 20:18:59,2019-02-28 20:46:00,6.0
343558,SP,7143,RS,93042,standard,1,4,done,2019-01-16,2019-02-20 20:18:59,2019-02-28 20:46:00,6.0
409512,SP,19027,SC,89940,standard,1,1,done,2019-01-20,2019-01-21 16:34:00,2019-02-26 14:05:00,25.0


Tenemos entonces 11863 filas del dataset (2.37%) que son iguales, dato a dato, a alguna otra fila. La pregunta que corresponde hacerse es: ¿Es suficiente saber que dos filas son iguales dato a dato para considerarlas iguales, o podríamos considerar por error que son datos iguales cuando corresponden a envíos distintos?

A priori, creemos que no hay una respuesta que podamos dar que tenga una certidumbre razonable para tomar una decisión, sin contar con el aporte de información de expertos en el dominio (esto es, personas que sepan cómo estos datos fueron tomados o generados, qué tipo de error tiene cada uno de los datos, cuáles datos son generados automáticamente y cuáles manualmente, etc.) o información en el dataset que sean indicadores determinado para esta unicidad (como por ejemplo, un identificador -una o más variables del dataset- declarado como tal).

Intentaremos sumar un poco más de información a partir de la exploración del dataset y de los (potencialmente) duplicados para tomar una decisión informada:

1. Lo primero que vemos es que tanto **date_sent** como **date_received** tienen precisión al segundo. Podríamos asumir que aquellas entradas con el valor del segundo en 0 para algunos de estos campos podrían responder a que la forma de carga toma información hasta el minuto, pero vemos ocurrencias de entradas con segundos distintos de 0, con lo cual esta asunción no tendría mucho sustento (o al menos no lo tendría para muchos de los casos).
Con lo cual, la probabilidad de que dos paquetes que salen del mismo estado, van al mismo estado, usan el mismo correo, y llegan y son recibidos a una misma fecha (con precisión de segundos, más allá de si este dato en particular se refiere al momento de carga o al momento preciso de envío o recepción) consideramos que es realmente baja. Este punto apoyaría la moción de considerar como repetidas estas entradas.

2. Para el caso de envíos con **quantity** mayor a 1, esta moción se reforzaría, ya que los envíos con más de un paquete son una parte minoritaria de los envíos, y la probabilidad de que no fuera un duplicado sería aún más baja.

3. Mismo análisis que el punto anterior se puede hacer para el caso de **shipment_type** super express.

A partir de este análisis, creemos que tiene sentido considerar a las filas que comparten el mismo valor para todas sus columnas como repetidas y no como envíos distintos. O, en todo caso, considerarlos como datos que no son confiables para la problemática que estamos intentando atacar.

Siendo que estas filas pueden estar repetidas más de dos veces, intentemos cuantificar de cuántas estamos hablando:

In [10]:
ds[ds.duplicated()].sort_values('date_created', ascending=True)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
436591,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
172193,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
159612,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
429821,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
434070,SP,6422,PR,82600,express,1,0,done,2018-12-28,NaT,NaT,NaN
343558,SP,7143,RS,93042,standard,1,4,done,2019-01-16,2019-02-20 20:18:59,2019-02-28 20:46:00,6.0
409512,SP,19027,SC,89940,standard,1,1,done,2019-01-20,2019-01-21 16:34:00,2019-02-26 14:05:00,25.0
333068,SP,15440,PA,68275,standard,1,1,done,2019-01-21,2019-01-22 12:30:00,2019-02-28 16:31:00,26.0
414708,SP,13977,RJ,28860,standard,1,4,done,2019-01-22,2019-03-12 23:34:36,2019-03-25 18:34:00,9.0
438991,SP,5813,MG,38730,standard,1,1,failed,2019-01-25,2019-01-28 12:42:00,2019-03-08 17:19:00,26.0


Listando sólo una ocurrencia de las filas repetidas (en este caso la 1era, de acuerdo a cómo funciona *duplicated*) tenemos 6483 filas únicas del original 11863 que incluyen todas las repetidas. Esto nos da un total de 5380 filas que estaríamos considerando repetidas y debemos quitar del dataset como parte de nuestro proceso de curación.

### Datos faltantes

Lo siguiente que vamos a revisar, como lo hicimos en el práctico anterior, son los datos faltantes (para cualquiera de las columnas del dataset). Intentaremos listarnos, buscar una explicación de los mismos, y decidir si (y cómo) lidiamos con ellos:

In [11]:
ds.isnull().sum()

sender_state          0
sender_zipcode        0
receiver_state        0
receiver_zipcode      0
shipment_type         0
quantity              0
service               0
status                0
date_created          0
date_sent            29
date_visit          602
shipment_days       602
dtype: int64

Pandas reconoce 1233 valores faltantes, en las variables **date_sent** (29), **date_visit** (602) y **shipment_days** (602). Esperaríamos a priori que:
1. Las filas con **shipment_days** faltantes y las filas con **date_visit** faltantes sean las mismas;
1. Que los valores faltantes se encuentren en envíos con un **status** que explique este faltante:

In [12]:
filter_1 = ds.shipment_days.isnull() | ds.date_visit.isnull()
len(ds[filter_1])

602

Validamos el primer supuesto. Veamos el segundo:

In [13]:
filter_2 = ds.date_sent.isnull() | ds.date_visit.isnull()
ds[filter_2].status.unique()

array(['sent', 'done', 'cancelled', 'set'], dtype=object)

No validamos el segundo supuesto, llama la atención los envíos en "done" con estas condiciones, con lo que podemos concluir que pueden ser anómalos:

In [14]:
ds[filter_2][ds.status == 'done']

/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
17843,SP,3437,SP,8742,express,1,4,done,2019-04-26,NaT,NaT,NaN
64821,SP,6422,PR,82600,express,1,0,done,2018-12-02,NaT,NaT,NaN
66437,SP,3634,SP,4542,express,1,4,done,2019-04-27,NaT,NaT,NaN
72863,SP,6422,BA,44006,express,1,0,done,2019-03-06,NaT,NaT,NaN
133877,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
136452,SP,8032,SP,6268,express,1,0,done,2019-02-26,NaT,NaT,NaN
148096,SP,6422,PR,82600,express,1,0,done,2019-01-02,NaT,NaT,NaN
159612,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
172193,SP,6422,PR,82600,express,1,0,done,2018-12-06,NaT,NaT,NaN
212770,SP,6422,PR,82600,express,2,0,done,2018-12-28,NaT,NaT,NaN


Intentaremos ver si existen otros valores "faltantes" que Pandas no esté reconociendo como tales:

In [15]:
# ToDo: Esto se puede hacer un poco más prolijo
for col in ds:
    print (ds[col].unique())

['SP']
[ 1039  3004  3262 ... 15195  5444 18304]
['SP' 'PB' 'RJ' 'AP' 'RS' 'MG' 'DF' 'MT' 'PR' 'GO' 'ES' 'SC' 'TO' 'BA'
 'PI' 'MS' 'PA' 'AC' 'PE' 'MA' 'RN' 'CE' 'AL' 'RO' 'AM' 'SE' 'RR']
[14500 58670 22723 ... 98788 72584  9612]
['standard' 'express' 'super']
[1 2 4 3 5]
[ 1  2  9  0  4  6  3  5  8  7 10 12 11]
['done' 'failed' 'sent' 'cancelled' 'set']
['2019-03-10T00:00:00.000000000' '2019-03-08T00:00:00.000000000'
 '2019-02-21T00:00:00.000000000' '2019-02-25T00:00:00.000000000'
 '2019-02-15T00:00:00.000000000' '2019-03-19T00:00:00.000000000'
 '2019-02-26T00:00:00.000000000' '2019-02-20T00:00:00.000000000'
 '2019-02-10T00:00:00.000000000' '2019-03-05T00:00:00.000000000'
 '2019-03-25T00:00:00.000000000' '2019-03-18T00:00:00.000000000'
 '2019-03-22T00:00:00.000000000' '2019-02-06T00:00:00.000000000'
 '2019-03-11T00:00:00.000000000' '2019-03-13T00:00:00.000000000'
 '2019-03-04T00:00:00.000000000' '2019-03-06T00:00:00.000000000'
 '2019-03-09T00:00:00.000000000' '2019-02-27T00:00:00.00000

A simple vista, pareciera que todos los valores faltantes están considerados en el análisis anterior.

Dicho esto, y con el contexto de la problemática que deseamos resolver, creemos que es conveniente eliminar las filas con datos faltantes: responden a datos que no nos servirán de información para la estimación de envío (son envíos que no llegaron, por diversas razones) o son datos que consideramos anómalos para el dataset. Cualquier política para completar esos datos con algún valor distinto (persistiendo la fila en cuestión) creemos que nos llevaría a tener información no confiable en el dataset.

### Datos anómalos

Más allá de los valores anómalos mencionados en la sección anterior, intentaremos identificar otras anomalías o inconsistencias en el data set.

**ToDo: Acá copiaría exactamente lo que hicimos el práctico anterior ("Análisis de shipment_days anómalos" y "Análisis de fechas anómalas")**


### Análisis de fechas anómalas (Parte I)


Las 3 variables de fechas que tenemos en este dataset, y los supuestos que proponemos para ellas, son:

- **date_created**: Corresponde a la fecha de creación del envío.
  - Es una fecha que se crea automáticamente a través de algunos de los procesos de venta de MercadoLibre.
  - Sólo nos interesa la fecha y no la hora de creación
- **date_sent**: Corresponde a la fecha y hora en que alguno de los correos cargó para el envío en cuestión, que comenzó el proceso de envío
 - Consideraremos **date_sent** como la fecha en que el envío está a cargo del correo y no necesariamente cuando el envío "salió" del correo (más bien cuando "entró" al correo, que es cuando el proceso de handling llega a su fin).
 - Puede crearse "automáticamente" cuando el vendedor deja el contenido del envío en el correo.
  - Puede también crearse "manualmente", pensando en lugares de recepción de correos sin la tecnología suficiente para que esta fecha sea generada automáticamente. Es posible entonces que sea necesario "aceptar" algunas fechas no precisas como válidas.
 * Por los supuestos en **date_created** y las anteriores para **date_sent**, esta fecha debe necesariamente ser posterior a **date_created**
- **date_visit**: Corresponde a la fecha y hora en que alguno de los correos cargó en envío en cuestión como entregado.
  - Puede crearse "automáticamente" cuando el cartero utiliza tecnología (lector de código de barras o aplicación movil, por ejemplo).
  - Puede crearse "manualmente", pensando en que el cartero, al terminar un ciclo o jornada, provee de la información de todos los envíos entregados en ese ciclo o jornada.
  - Por los supuestos en **date_created**, **date_sent** y los anteriores de **date_visit**, esta fecha debe neceariamente ser posterior a **date_created**. Se espera también que sea posterior a **date_sent** también, pero deberemos tomar una decisión en estos casos en función de análisis del dataset.

In [16]:
m_date_created_after_date_sent = (ds.date_created >= ds.date_sent)
m_date_created_before_date_sent = (ds.date_created < ds.date_sent)

m_date_created_after_date_visit = (ds.date_created >= ds.date_visit)
m_date_created_before_date_visit = (ds.date_created < ds.date_visit)

m_date_created_invalid = m_date_created_after_date_sent | \
                         m_date_created_after_date_visit

ds[m_date_created_invalid].describe()

,sender_zipcode,receiver_zipcode,quantity,service,shipment_days
count,194.000000,194.000000,194.000000,194.000000,179.000000
mean,7733.309278,41872.061856,1.237113,5.969072,-8.508380
std,4884.848489,30855.877916,0.589602,1.862939,34.751828
min,1039.000000,1021.000000,1.000000,2.000000,-77.000000
25%,3120.750000,13930.000000,1.000000,4.000000,-32.500000
50%,7411.000000,29108.500000,1.000000,7.000000,0.000000
75%,9961.000000,71970.500000,1.000000,7.000000,17.000000
max,19160.000000,97502.000000,4.000000,10.000000,38.000000


Tenemos 194 casos donde los datos no siguen los supuestos planteados. Por lo tanto, consideramos que esta información anómala no es confiable y deberíamos eliminarla del dataset.

In [17]:
m_date_sent_after_date_visit = ds.date_sent >= ds.date_visit
m_date_sent_before_date_visit = ds.date_sent < ds.date_visit

ds[m_date_sent_after_date_visit].describe()

,sender_zipcode,receiver_zipcode,quantity,service,shipment_days
count,2407.000000,2407.000000,2407.000000,2407.000000,2407.000000
mean,7066.651849,12403.598671,1.283756,2.230577,-1.218114
std,4860.648858,17464.272237,0.670354,2.867468,9.042260
min,1001.000000,1005.000000,1.000000,0.000000,-77.000000
25%,3026.000000,4107.500000,1.000000,0.000000,0.000000
50%,5373.000000,6663.000000,1.000000,0.000000,0.000000
75%,11017.500000,13214.000000,1.000000,5.000000,0.000000
max,19970.000000,95300.000000,5.000000,10.000000,0.000000


Tenemos 2407 casos en donde se consignó que el paquete entró al proceso de envío después de la fecha de consignación de llegada. No representan una cantidad significativa de casos como para analizar si tiene sentido sacarlos o dejarlos del dataset, por lo que consideramos que es información no confiable que podemos quitar.

### Análisis de fechas anómalas (Parte II)

In [18]:
dataset.describe(include='datetime64')

,date_created,date_sent,date_visit
count,499952,499923,499350
unique,144,81354,55668
top,2019-03-05 00:00:00,2019-03-11 15:53:00,2019-03-11 22:00:00
freq,20456,113,621
first,2018-09-06 00:00:00,2018-10-23 13:57:00,2019-02-25 05:36:00
last,2019-06-28 00:00:00,2019-03-29 01:46:20,2019-04-02 15:35:00


En el práctico anterior habiamos observado lo siguiente:

* La variables **date_created** solo tiene 144 valores distintos y no considera horas (todas están seteadas a las 00:00 hs) esto implica que estamos considerando un total de 144 días distintos.
* La variable **date_created**  contienen fechas que caen fuera del intervalo temporal considerado y más aún, que representa una fecha futura (Junio de 2019).
* Hay una cierta variabilida entre los mínimos de las 3 fechas, podría servir analizar los estados de las fechas fuera del periodo de **date_visit** para ver si no son anómalos.
* En la variable **date_visit** el dato más frecuente se da a las 22.00 hs, posiblemente fuera del horario de atención de los correos, con lo cual este dato es bastante extraño en un principio. Lo mismo sucede con el horario 05:36.

Teniendo en cuenta la problemática que queremos resolver, la primera observación cobra cierto sentido pues el día de creación del envío resulta mucho más importante que la hora en si misma. Además, considerando que la respuesta del problema en un principio debe consistir en dar un intervalo de días, las franjas horarias podrían no ser tenidas en cuenta a priori.

Con respecto a la segunda observación podríamos intentar determinar la cantidad de fechas que caen fuera del intervalo Octubre de 2018-Abril de 2019 y al mismo tiempo observar la tendencia a medida que pasan los meses de las variables **date_created**, **date_sent** y **date_visited**.



In [19]:
dataset['date_created'].groupby(
    [dataset['date_created'].dt.year.rename('year'),
     dataset['date_created'].dt.month.rename('month')]).agg({'count'})


count
year month        
2018 9           1
     10          1
     11          5
     12         45
2019 1        3715
     2      180959
     3      315080
     4          35
     6         111

**Observaciones:**

* Tenemos una fecha de creación en el mes de septiembre y 111 fechas en el mes de Junio . Todas ellas caen fuera del intervalo considerado.

* La mayor concentración en cuanto a fechas de creación de envíos se dió en la franja enero-marzo, donde marzo cuenta con más de la mitad de los datos. 

**Preguntas y comentarios a tener en cuenta:**

1. ¿Es normal que en un intervalo de 6 meses los datos se concentren mayormente en un mes o dos?
2. ¿Podemos pensar que los datos de septiembre de 2018 y julio de 2019 hayan tenido que ver con errores de carga de los datos y sean efectivamente datos que se encuentran en el intervalo considerado? ¿O efectivamente serán datos que tenemos que eliminar de nuestro conjunto de análisis?
3. Los 111 datos en el mes de junio resultan bastante extraños y quizás merecen un análisis más detallado.

In [20]:
dataset['date_sent'].groupby(
    [dataset['date_sent'].dt.year.rename('year'),
     dataset['date_sent'].dt.month.rename('month')]).agg({'count'})


count
year   month        
2018.0 10.0        1
       11.0        2
       12.0       11
2019.0 1.0      2393
       2.0    152315
       3.0    345201

**Observaciones:**

No se observan datos fuera del intervalo considerando pero si  una gran concentración en el intervalo enero-marzo.



In [21]:
dataset['date_visit'].groupby(
    [dataset['date_visit'].dt.year.rename('year'),
     dataset['date_visit'].dt.month.rename('month')]).agg({'count'})


count
year   month        
2019.0 2.0     27960
       3.0    470034
       4.0      1356

**Observaciones:**

* No se observan datos anómalos en la variable **date_visit**.
* Resulta extraño que tengamos fecha de visita solo en los meses de febrero, marzo y abril cuando nuestro intervalo temporal es más grande
* Estos datos merecen un análisis más detallado.



### Análisis de la variable date_created

Ahora vamos a analizar las 111 fechas de creación del mes de junio y el valor del mes de septiembre para ver si encontramos alguna anomalia.

In [22]:
date_created_septiembre = dataset['date_created'].map(lambda x: x.month) == 9
dataset[date_created_septiembre]

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
293012,SP,6422,RS,91180,express,1,0,done,2018-09-06,NaT,NaT,NaN


Consideramos que este dato está mal cargado y su presencia no aporta valor a nuestro análisis. Por lo tanto procederemos a eliminarlo.

In [23]:
dataset.drop(dataset[dataset['date_created'].map(lambda x: x.month) == 9].index, inplace=True)

/Users/nsalina/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
dataset[date_created_septiembre] #Corroboramos que se haya eliminado

/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days


In [25]:
date_created_junio = dataset['date_created'].map(lambda x: x.month) == 6
dataset[date_created_junio]

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
3034,SP,17512,PE,53260,standard,1,4,sent,2019-06-21,2019-02-06 05:54:00,NaT,NaN
14265,SP,2350,RJ,20261,express,1,7,done,2019-06-27,2019-02-06 05:57:00,2019-03-01 11:31:00,17.0
18211,SP,7756,SP,13214,express,1,7,done,2019-06-27,2019-02-06 05:00:00,2019-03-01 09:08:00,17.0
27700,SP,5034,GO,74565,standard,1,7,done,2019-06-26,2019-02-06 10:56:00,2019-03-01 11:54:00,17.0
28540,SP,9060,MA,65050,standard,1,4,done,2019-06-20,2019-02-06 10:00:00,2019-03-01 12:02:00,17.0
33158,SP,16204,MG,31910,standard,2,7,done,2019-06-27,2019-02-06 04:03:00,2019-03-01 16:43:00,17.0
38127,SP,2460,SP,13335,express,1,7,done,2019-06-26,2019-03-06 01:18:00,2019-03-07 14:03:00,0.0
42758,SP,1042,SP,11070,express,1,4,done,2019-06-25,2019-02-06 05:01:00,2019-03-01 16:09:00,17.0
43607,SP,6422,RS,91750,standard,1,7,done,2019-06-19,2019-02-06 04:41:00,2019-03-01 14:30:00,17.0
45731,SP,14405,ES,29164,standard,1,7,done,2019-06-22,2019-03-06 03:02:00,2019-03-07 04:27:00,0.0


Si bien estos datos caen fuera de los supuestos en cuanto a que la fecha de creación debe ser anterior a la fecha de envío, la inspección de los datos de arriba  nos inclina a pensar que los datos con fecha de creación en junio de 2019 en realidad corresponden a datos del mes de enero del mismo año. De esa manera, las columnas de las fechas parecerian tener más coherencia y cumplir con los supuestos planteados. Bajo esta hipótesis nos encontramos frente a la diyuntiva de si eliminamos esta información o imputamos los datos haciendo que su fecha de creación sea  enero de 2019.

Si uno se guía por los datos como vienen la primera opción sería estas filas (en cuanto a cantidad no afectaría a la cantidad total) sin embargo podemos tomar la jugada arriesgada y "modificar" nuestros datos subsanando de esta manera en posible error producido durante la carga de los datos. 

A continuación reemplazaremos los datos de creación de junio de 2019 por enero de 2019.

## Dataset limpio

#ToDo 
(habría que charlar esta parte en base a la parte II de arriba)

En función de lo visto en las secciones anteriores, generaremos un nuevo dataset con las siguientes características:

- Sin datos "potencialmente duplicados" (ver "Datos duplicados")
- Sin datos nulos (ver "Datos faltantes")
- Sin **shipment_days** negativos (ver "Análisis de shipment_days anómalos")
- Sin envíos creados después de ser enviados (ver "Análisis de fechas anómalas")
- Sin envíos creados después de ser recibidos (ver "Análisis de fechas anómalas")
- Sin envíos enviados después de ser recibidos (ver "Análisis de fechas anómalas")

In [26]:
# Generamos un nuevo dataset "limpio"

# 1. Sin datos "potencialmente duplicados"
dataset = ds.drop_duplicates()

# 2. Sin datos nulos
dataset = dataset.dropna()

# 3. Sin shipment_days negativos
dataset = dataset[m_shipment_days_positive]

# 4. Sin envíos creados después de ser enviados
dataset = dataset[m_date_created_before_date_sent]

# 5. Sin envíos creados después de ser recibidos
dataset = dataset[m_date_created_before_date_visit]

# 6. Sin envíos enviados después de ser recibidos
dataset = dataset[m_date_sent_before_date_visit]

dataset.to_csv(RELATIVE_PATH + "data_sample_corrected_cleaned.csv", sep=',', index=False)
dataset.describe()

/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/nsalina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,sender_zipcode,receiver_zipcode,quantity,service,shipment_days
count,490439.000000,490439.000000,490439.000000,490439.000000,490439.000000
mean,8317.085674,40114.240511,1.278581,1.496551,5.409976
std,5091.026212,30968.774147,0.662111,1.809156,4.981929
min,1001.000000,1001.000000,1.000000,0.000000,0.000000
25%,3685.000000,13171.000000,1.000000,0.000000,2.000000
50%,7400.000000,31340.000000,1.000000,1.000000,4.000000
75%,13290.000000,70732.000000,1.000000,2.000000,8.000000
max,19970.000000,99990.000000,5.000000,12.000000,90.000000


Resultan 490.439 entradas (de las 500.000 originales), con lo cual hemos dejado en el camino menos del 2% del dataset original, un número que, por la confianza que nos transmiten esos datos, podemos anticipar que es razonable.

## Pregunta 3

Para convertir datos categóricos en numéricos se pueden usar, entre otros, los siguientes dos métodos:
* **One-Hot encoding:** traduce cada valor categórico en una columna independiente con valor 1 en los registros que tenían dicha categoría. Este método, si bien no tiene problemas con los "pesos" que pueden heredar las categorías, puede presentar problemas cuando el número total de categorías es grande, ya que los datos se codifican en grandes dimensiones.
* **Binary Encoding:** toma el índice ordinal de las categorías y las codifica de forma binaria, donde los bits de esa codificación se encuentran separados en columnas. La ventaja que tiene respecto al método anterior es que los datos se codifican en menores dimensiones.

In [27]:
dataset['receiver_state'] = pd.Categorical(dataset['receiver_state'])


In [28]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490439 entries, 0 to 499999
Data columns (total 12 columns):
sender_state        490439 non-null object
sender_zipcode      490439 non-null int64
receiver_state      490439 non-null category
receiver_zipcode    490439 non-null int64
shipment_type       490439 non-null object
quantity            490439 non-null int64
service             490439 non-null int64
status              490439 non-null object
date_created        490439 non-null datetime64[ns]
date_sent           490439 non-null datetime64[ns]
date_visit          490439 non-null datetime64[ns]
shipment_days       490439 non-null float64
dtypes: category(1), datetime64[ns](3), float64(1), int64(4), object(3)
memory usage: 45.4+ MB


**One-Hot encoding**

In [29]:
dfDummies = pd.get_dummies(dataset['receiver_state'], prefix = 'category')


In [30]:
dataset_dummies = pd.concat([dataset,dfDummies], axis=1)
dataset_dummies.sample(10)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,...,category_PR,category_RJ,category_RN,category_RO,category_RR,category_RS,category_SC,category_SE,category_SP,category_TO
215755,SP,9540,MG,34800,standard,1,1,done,2019-02-26,2019-02-26 17:06:00,...,0,0,0,0,0,0,0,0,0,0
417807,SP,9760,SP,6381,express,1,0,done,2019-03-07,2019-03-08 10:48:00,...,0,0,0,0,0,0,0,0,1,0
295496,SP,19024,SP,13070,standard,1,1,done,2019-02-23,2019-02-25 12:36:00,...,0,0,0,0,0,0,0,0,1,0
283628,SP,9390,MG,34002,standard,1,1,done,2019-03-06,2019-03-07 15:20:00,...,0,0,0,0,0,0,0,0,0,0
93489,SP,3144,SP,11055,express,1,4,done,2019-03-01,2019-03-04 23:36:12,...,0,0,0,0,0,0,0,0,1,0
230248,SP,4136,SP,14540,express,4,0,done,2019-03-07,2019-03-07 18:49:00,...,0,0,0,0,0,0,0,0,1,0
467937,SP,13290,SP,18603,super,3,6,done,2019-03-16,2019-03-16 18:55:54,...,0,0,0,0,0,0,0,0,1,0
55111,SP,14405,RJ,27163,standard,1,2,done,2019-03-14,2019-03-20 02:46:00,...,0,1,0,0,0,0,0,0,0,0
370035,SP,9991,AM,69095,express,1,0,done,2019-03-15,2019-03-15 15:46:00,...,0,0,0,0,0,0,0,0,0,0
38606,SP,6422,MG,37590,standard,2,2,done,2019-03-08,2019-03-11 15:34:00,...,0,0,0,0,0,0,0,0,0,0


**Binary Encoding**

In [31]:
dataset_ce = dataset.copy()

encoder = ce.BinaryEncoder(cols=['receiver_state'])
dataset_binary = encoder.fit_transform(dataset_ce)

dataset_binary.head()

,sender_state,sender_zipcode,receiver_state_0,receiver_state_1,receiver_state_2,receiver_state_3,receiver_state_4,receiver_state_5,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,shipment_days
0,SP,1039,0,1,1,0,1,0,14500,standard,1,1,done,2019-03-10,2019-03-11 14:34:00,2019-03-18 09:52:00,5.0
1,SP,3004,0,0,1,1,1,1,58670,standard,1,1,done,2019-03-08,2019-03-09 10:48:00,2019-03-18 15:16:00,5.0
2,SP,3262,0,1,0,0,1,1,22723,standard,1,2,done,2019-02-21,2019-02-23 02:02:16,2019-03-01 21:27:00,4.0
3,SP,12237,0,1,1,0,1,0,18682,standard,1,2,done,2019-02-25,2019-02-28 03:05:36,2019-03-07 15:01:00,2.0
4,SP,9350,0,0,0,1,0,0,68908,standard,1,1,done,2019-02-15,2019-02-15 11:28:00,2019-03-11 13:18:00,13.0
